In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet, VGG16
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import cv2
import matplotlib.pyplot as plt
from scipy.integrate import simps
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN 
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
import visualkeras
import matplotlib.pyplot as plt
from mtcnn import MTCNN  
from matplotlib.patches import Rectangle, Circle

2024-10-03 21:47:45.239090: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 21:47:45.239151: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 21:47:45.241435: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 21:47:45.456456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.14.0'

In [ ]:
path="/home/gourav/Desktop/vss/capstone/best_model_mobile.h5"
img_size=224
beep_sound_file = "/home/gourav/Desktop/vss/capstone/beep-01a.wav"
model=tf.keras.models.load_model(path)

In [ ]:
face_cascade=cv2.CascadeClassifier(r'/home/gourav/Desktop/vss/capstone/haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
counter=0
while True:
    ret,frame=cap.read()
    
    if not ret:
        print("Error")
        break
    
    eyes_cascade=cv2.CascadeClassifier(r'/home/gourav/Desktop/vss/capstone/haarcascade_eye_tree_eyeglasses.xml')
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    eyes=eyes_cascade.detectMultiScale(gray,1.1,4)
    for (x,y,w,h) in eyes:
        roi_gray=gray[y:y+h,x:x+h]
        roi_color=frame[y:y+h,x:x+h]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        eyess=eyes_cascade.detectMultiScale(roi_gray)
        if len(eyess) == 0:
            print("eyes not found")
        else :
            for (ex,ey,ew,eh) in eyess:
                eyes_roi=roi_color[ey:ey+eh,ex:ex+ew]

    final_image=cv2.resize(eyes_roi,(img_size,img_size))
    final_image=np.array(final_image).reshape(1,img_size,img_size,3)# (one image 224,224,3)
    final_image=final_image/255.0

    prediction=model.predict(final_image)
    
    if (prediction>0.9):
           
        x1,y1,w1,h1=0,0,175,75
        cv2.rectangle(frame,(x1,y1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,"Active",(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    else:
        counter=counter+1
        if counter>5:
            x1,y1,w1,h1=0,0,175,75
            for i in range(2):
                cv2.rectangle(frame,(x1,y1),(x1+w1,y1+h1),(0,0,0),-1)
                cv2.putText(frame,"Sleep alert !!! ",(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            counter=0
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray,1.1,4)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        
    cv2.imshow("driver drowsiness detection",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()
